### GDAL commands
1. Convert individual 1 band image (mask image) to PNG
```
gdal_translate -of PNG -scale -co worldfile=no mask2.tif output6.png
```

### View any image details

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import skimage.io as io

image_path = 'output6.png'
image = io.imread(image_path)
image = image / 255   # normalize to 0-1

io.imshow(image)
print("SHAPE: ",image.shape)
print("UNIQUE VALUES: ",np.unique(image))
print("NUMBER OF CLASSES: ",len(np.unique(image)))

### Converting grayscale to final mask

In [ ]:
from PIL import Image
input_image_path = "output3.png"
output_image_path = "output.png"
image = Image.open(input_image_path)
arr = np.array(image) / 255.0  # Normalize pixel values to [0, 1]

#input image details
print(arr.shape)
print(np.unique(arr))
print(len(np.unique(arr)))

#convert image to 3 classes (even if it is redundant in some cases - it is a safeguard)
THRESHOLD1 = 0.45 #NDVI: non-tree and sparse tree
THRESHOLD2 = 0.7  #NDVI: sparse tree and dense tree
CLASS1 = 0 #white
CLASS2 = 1 #grey
CLASS3 = 2  #black
arr2 = arr.copy()
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        if arr[i][j] < THRESHOLD1:
            arr2[i][j] = CLASS1
        elif arr[i][j] >= THRESHOLD1 and arr[i][j] < THRESHOLD2:
            arr2[i][j] = CLASS2
        else:
            arr2[i][j] = CLASS3

#output image details
print(arr2.shape)
print(np.unique(arr2))
print(len(np.unique(arr2)))

# Display the image
image = Image.fromarray(arr2)
plt.imshow(image)

# save the image
mask_image = Image.fromarray(arr2.astype(np.uint8))
mask_image.save(output_image_path)

### Converting Grayscale to Green RGB

In [ ]:
input_image_path = "final2.png" 
output_image_path = "finale2.png"
image = Image.open(input_image_path)
arr = np.array(image)

#input image details
print(arr.shape)
print(np.unique(arr))
print(len(np.unique(arr)))

# convert the grayscale to  green RGB
CLASS1 = 255 #white
CLASS2 = 150 #grey
CLASS3 = 25  #black

WHITE = [255, 255, 255]
LIGHT_GREEN = [144, 238, 144]
DARK_GREEN = [0, 100, 0]

arr2 = np.zeros((arr.shape[0], arr.shape[1], 3), dtype=np.uint8)
print(arr2.shape)
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        if arr[i][j] == CLASS1:
            arr2[i][j] = WHITE
        elif arr[i][j] == CLASS2:
            arr2[i][j] = LIGHT_GREEN
        else:
            arr2[i][j] = DARK_GREEN

#output image details
print(arr2.shape)
print(len(np.unique(arr2)))
print(np.unique(arr2))

# Display the image
image = Image.fromarray(arr2)
plt.imshow(image)

# save the image
mask_image = Image.fromarray(arr2.astype(np.uint8))
mask_image.save(output_image_path)

### Seperate Image and Masks into folders
- Keep all the images in `ChangeDetection/Images`

In [ ]:
import os
import shutil

def move_mask_files(source_folder, destination_folder):
    """
    Move all files starting with 'm_' from the source folder to the destination folder.

    :param source_folder: Path to the folder containing the files.
    :param destination_folder: Path to the folder where the files will be moved.
    """
    # Ensure the destination folder exists
    os.makedirs(destination_folder, exist_ok=True)
    
    # Iterate through all files in the source folder
    for file_name in os.listdir(source_folder):
        if file_name.startswith("m_"):
            # Build full paths
            source_path = os.path.join(source_folder, file_name)
            destination_path = os.path.join(destination_folder, file_name)
            
            # Move the file
            shutil.move(source_path, destination_path)
            print(f"Moved: {file_name}")

# Example usage:
source_folder = "ChangeDetection/Images"
destination_folder = "ChangeDetection/Masks"

move_mask_files(source_folder, destination_folder)

### Seperate 2019 and 2024

In [ ]:
import os
import shutil

def move_mask_files(source_folder, destination_folder):
    """
    Move all files starting with 'm_' from the source folder to the destination folder.

    :param source_folder: Path to the folder containing the files.
    :param destination_folder: Path to the folder where the files will be moved.
    """
    # Ensure the destination folder exists
    os.makedirs(destination_folder, exist_ok=True)
    
    # Iterate through all files in the source folder
    for file_name in os.listdir(source_folder):
        if file_name.endswith("2024.tif"):
            # Build full paths
            source_path = os.path.join(source_folder, file_name)
            destination_path = os.path.join(destination_folder, file_name)
            
            # Move the file
            shutil.move(source_path, destination_path)
            print(f"Moved: {file_name}")

# Example usage:
source_folder = "ChangeDetection/Images/T2019"
destination_folder = "ChangeDetection/Images/T2024"

move_mask_files(source_folder, destination_folder)

### Generating Change Detection ground truths

#### Case 1: Vegetation Increase and Decrease

In [ ]:
import os
from collections import Counter
import numpy as np
import rasterio
import csv

def classify_veg_increase_decrease_classes(mask_2019_path, mask_2024_path, output_path, csv_writer):
    """
    Classify land-use changes into 3 classes, calculate area in km^2, and save to a CSV file.
    """
    # Read 2019 mask
    with rasterio.open(mask_2019_path) as src_2019:
        mask_2019 = src_2019.read(1)
        profile = src_2019.profile

    # Read 2024 mask
    with rasterio.open(mask_2024_path) as src_2024:
        mask_2024 = src_2024.read(1)

    # Initialize the output classification mask
    output_mask = np.zeros_like(mask_2019, dtype=np.uint8)

    # Classification logic
    output_mask[(mask_2019 == mask_2024)  
                | ((mask_2019 == 3) & (mask_2024 == 2))
                | ((mask_2019 == 2) & (mask_2024 == 3))] = 0  # No change (including water)
    output_mask[((mask_2019 == 1) & (mask_2024 == 3)) | ((mask_2019 == 1) & (mask_2024 == 2))] = 1  # Vegetation increase
    output_mask[((mask_2019 == 3) & (mask_2024 == 1)) | ((mask_2019 == 2) & (mask_2024 == 1))] = 2  # Vegetation decrease

    # Calculate pixel counts for each class
    pixel_counts = Counter(output_mask.flatten())

    # Calculate area for each class in km^2
    pixel_area_km2 = (10 * 10) / (1000 * 1000)  # Each pixel represents 100 m^2 converted to km^2
    area_by_class = {class_id: count * pixel_area_km2 for class_id, count in pixel_counts.items()}

    # Prepare data for CSV
    mask_name = os.path.basename(mask_2019_path).replace("_2019.tif", "")
    class_0_area = area_by_class.get(0, 0)
    class_1_area = area_by_class.get(1, 0)
    class_2_area = area_by_class.get(2, 0)

    csv_writer.writerow([mask_name, class_0_area, class_1_area, class_2_area])

    # Save the output classification mask to a new TIFF file
    profile.update(dtype=rasterio.uint8, count=1)
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(output_mask, 1)

    print(f"Processed {mask_name}: Saved output mask and areas.")

def process_all_masks(input_folder, output_folder, csv_file):
    """
    Process all masks from the input folder, save the output masks to the output folder,
    and log areas to a CSV file.
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Open CSV file for writing
    with open(csv_file, mode="w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Mask Name", "Class 0 Area", "Class 1 Area", "Class 2 Area"])  # Header row

        # Fetch all 2019 and 2024 masks
        mask_files_2019 = [f for f in os.listdir(input_folder) if "_2019.tif" in f]
        mask_files_2024 = [f for f in os.listdir(input_folder) if "_2024.tif" in f]

        # Match 2019 and 2024 masks by state and city and direction
        for mask_2019 in mask_files_2019:
            city_state_direction = mask_2019.replace("_2019.tif", "")
            mask_2024 = f"{city_state_direction}_2024.tif"
            if mask_2024 in mask_files_2024:
                mask_2019_path = os.path.join(input_folder, mask_2019)
                mask_2024_path = os.path.join(input_folder, mask_2024)
                output_path = os.path.join(output_folder, f"cd1_{city_state_direction}.tif")
                
                classify_veg_increase_decrease_classes(mask_2019_path, mask_2024_path, output_path, csv_writer)

# Example usage
input_folder = "ChangeDetection/Masks"
output_folder = "ChangeDetection/cd1_Output"
csv_file = "ChangeDetection/cd1_area_summary.csv"
process_all_masks(input_folder, output_folder, csv_file)


#### Case 2: Building <-> Dense vegetation

In [ ]:
import os
from collections import Counter
import numpy as np
import rasterio
import csv

def classify_veg_increase_decrease_classes(mask_2019_path, mask_2024_path, output_path, csv_writer):
    """
    Classify land-use changes into 3 classes, calculate area in km^2, and save to a CSV file.
    """
    # Read 2019 mask
    with rasterio.open(mask_2019_path) as src_2019:
        mask_2019 = src_2019.read(1)
        profile = src_2019.profile

    # Read 2024 mask
    with rasterio.open(mask_2024_path) as src_2024:
        mask_2024 = src_2024.read(1)

    # Initialize the output classification mask
    output_mask = np.zeros_like(mask_2019, dtype=np.uint8)

    # Classification logic
    output_mask[(mask_2019 == mask_2024)  
                | ((mask_2019 == 3) & (mask_2024 == 2))
                | ((mask_2019 == 1) & (mask_2024 == 2)) 
                | ((mask_2019 == 2) & (mask_2024 == 3))
                | ((mask_2019 == 2) & (mask_2024 == 1))] = 0  # No change (including water and dense <-> sparse vegetation)
    output_mask[((mask_2019 == 1) & (mask_2024 == 3)) ] = 1  # Building to dense vegetation
    output_mask[((mask_2019 == 3) & (mask_2024 == 1)) ] = 2  # dense Vegetation to building

    # Calculate pixel counts for each class
    pixel_counts = Counter(output_mask.flatten())

    # Calculate area for each class in km^2
    pixel_area_km2 = (10 * 10) / (1000 * 1000)  # Each pixel represents 100 m^2 converted to km^2
    area_by_class = {class_id: count * pixel_area_km2 for class_id, count in pixel_counts.items()}

    # Prepare data for CSV
    mask_name = os.path.basename(mask_2019_path).replace("_2019.tif", "")
    class_0_area = area_by_class.get(0, 0)
    class_1_area = area_by_class.get(1, 0)
    class_2_area = area_by_class.get(2, 0)

    csv_writer.writerow([mask_name, class_0_area, class_1_area, class_2_area])

    # Save the output classification mask to a new TIFF file
    profile.update(dtype=rasterio.uint8, count=1)
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(output_mask, 1)

    print(f"Processed {mask_name}: Saved output mask and areas.")

def process_all_masks(input_folder, output_folder, csv_file):
    """
    Process all masks from the input folder, save the output masks to the output folder,
    and log areas to a CSV file.
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Open CSV file for writing
    with open(csv_file, mode="w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Mask Name", "Class 0 Area", "Class 1 Area", "Class 2 Area"])  # Header row

        # Fetch all 2019 and 2024 masks
        mask_files_2019 = [f for f in os.listdir(input_folder) if "_2019.tif" in f]
        mask_files_2024 = [f for f in os.listdir(input_folder) if "_2024.tif" in f]

        # Match 2019 and 2024 masks by state and city and direction
        for mask_2019 in mask_files_2019:
            city_state_direction = mask_2019.replace("_2019.tif", "")
            mask_2024 = f"{city_state_direction}_2024.tif"
            if mask_2024 in mask_files_2024:
                mask_2019_path = os.path.join(input_folder, mask_2019)
                mask_2024_path = os.path.join(input_folder, mask_2024)
                output_path = os.path.join(output_folder, f"cd2_{city_state_direction}.tif")
                
                classify_veg_increase_decrease_classes(mask_2019_path, mask_2024_path, output_path, csv_writer)

# Example usage
input_folder = "ChangeDetection/Masks"
output_folder = "ChangeDetection/cd2_Output"
csv_file = "ChangeDetection/cd2_area_summary.csv"
process_all_masks(input_folder, output_folder, csv_file)

#### Case 3: All classes (7 classes)

In [ ]:
import os
from collections import Counter
import numpy as np
import rasterio
import csv

def classify_veg_increase_decrease_classes(mask_2019_path, mask_2024_path, output_path, csv_writer):
    """
    Classify land-use changes into 7 classes, calculate area in km^2, and save to a CSV file.
    """
    # Read 2019 mask
    with rasterio.open(mask_2019_path) as src_2019:
        mask_2019 = src_2019.read(1)
        profile = src_2019.profile

    # Read 2024 mask
    with rasterio.open(mask_2024_path) as src_2024:
        mask_2024 = src_2024.read(1)

    # Initialize the output classification mask
    output_mask = np.zeros_like(mask_2019, dtype=np.uint8)

    # Classification logic
    output_mask[(mask_2019 == mask_2024)] = 0  # No change (including water)
    output_mask[(mask_2019 == 1) & (mask_2024 == 2)] = 1  # Building to sparse veg
    output_mask[(mask_2019 == 1) & (mask_2024 == 3)] = 2  # Building to dense veg
    output_mask[(mask_2019 == 2) & (mask_2024 == 3)] = 3  # Sparse to dense
    output_mask[(mask_2019 == 2) & (mask_2024 == 1)] = 4  # Sparse to building
    output_mask[(mask_2019 == 3) & (mask_2024 == 2)] = 5  # Dense to sparse
    output_mask[(mask_2019 == 3) & (mask_2024 == 1)] = 6  # Dense to building


    # Calculate pixel counts for each class
    pixel_counts = Counter(output_mask.flatten())

    # Calculate area for each class in km^2
    pixel_area_km2 = (10 * 10) / (1000 * 1000)  # Each pixel represents 100 m^2 converted to km^2
    area_by_class = {class_id: count * pixel_area_km2 for class_id, count in pixel_counts.items()}

    # Prepare data for CSV
    mask_name = os.path.basename(mask_2019_path).replace("_2019.tif", "")
    class_0_area = area_by_class.get(0, 0)
    class_1_area = area_by_class.get(1, 0)
    class_2_area = area_by_class.get(2, 0)
    class_3_area = area_by_class.get(3, 0)
    class_4_area = area_by_class.get(4, 0)
    class_5_area = area_by_class.get(5, 0)
    class_6_area = area_by_class.get(6, 0)


    csv_writer.writerow([mask_name, class_0_area, class_1_area, class_2_area, 
                         class_3_area, class_4_area, class_5_area, class_6_area])

    # Save the output classification mask to a new TIFF file
    profile.update(dtype=rasterio.uint8, count=1)
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(output_mask, 1)

    print(f"Processed {mask_name}: Saved output mask and areas.")

def process_all_masks(input_folder, output_folder, csv_file):
    """
    Process all masks from the input folder, save the output masks to the output folder,
    and log areas to a CSV file.
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Open CSV file for writing
    with open(csv_file, mode="w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Mask Name", "Class 0 Area", "Class 1 Area", "Class 2 Area", "Class 3 Area", 
                             "Class 4 Area", "Class 5 Area", "Class 6 Area"])  # Header row

        # Fetch all 2019 and 2024 masks
        mask_files_2019 = [f for f in os.listdir(input_folder) if "_2019.tif" in f]
        mask_files_2024 = [f for f in os.listdir(input_folder) if "_2024.tif" in f]

        # Match 2019 and 2024 masks by state and city and direction
        for mask_2019 in mask_files_2019:
            city_state_direction = mask_2019.replace("_2019.tif", "")
            mask_2024 = f"{city_state_direction}_2024.tif"
            if mask_2024 in mask_files_2024:
                mask_2019_path = os.path.join(input_folder, mask_2019)
                mask_2024_path = os.path.join(input_folder, mask_2024)
                output_path = os.path.join(output_folder, f"cd3_{city_state_direction}.tif")
                
                classify_veg_increase_decrease_classes(mask_2019_path, mask_2024_path, output_path, csv_writer)

# Example usage
input_folder = "ChangeDetection2/Masks"
output_folder = "ChangeDetection2/cd3_Output"
csv_file = "ChangeDetection2/cd3_area_summary.csv"
process_all_masks(input_folder, output_folder, csv_file)

### Split dataset into train test val

#### File structures
1. Before split - 
```
Dataset ->{ Images->{T2019,T2024}, Masks->{T2019,T2024},
    cd1_Output, cd2_Output, cd3_Output
}
```

2. After split - 
```
SplitDataset ->{
    train->Dataset, test->Dataset, val->Dataset
}
WHERE
Dataset->{
    Images->{T2019,T2024}, Masks->{T2019,T2024},
    cd1_Output, cd2_Output, cd3_Output
}
```

- Filenames <br>
    - Images = `Alabama_Birmingham_C_2019.tif` <br>
    - Masks = `m_Alabama_Birmingham_C_2019.tif` <br>
    - cd1_Output = `cd1_m_Alabama_Birmingham_C.tif` <br>

In [ ]:
import os
import shutil
from collections import defaultdict
import random

def split_dataset(dataset_dir, output_dir, train_count=1070, val_count=230, test_count=230):
    # Define directories
    images_2019 = os.path.join(dataset_dir, "Images", "T2019")
    images_2024 = os.path.join(dataset_dir, "Images", "T2024")
    masks_2019 = os.path.join(dataset_dir, "Masks", "T2019")
    masks_2024 = os.path.join(dataset_dir, "Masks", "T2024")
    cd1_output = os.path.join(dataset_dir, "cd1_Output")
    cd2_output = os.path.join(dataset_dir, "cd2_Output")
    cd3_output = os.path.join(dataset_dir, "cd3_Output")

    # Group files by city
    city_files = defaultdict(list)
    for filename in os.listdir(images_2019):
        if filename.endswith(".tif"):
            city = "_".join(filename.split("_")[:-1])  # Extract city (e.g., "Alabama_Birmingham_C")
            city_files[city].append(filename)

    # Shuffle cities for randomization
    cities = list(city_files.keys())
    random.shuffle(cities)

    # Split cities into train, val, and test with precise file counts
    train_cities, val_cities, test_cities = [], [], []
    train_files, val_files, test_files = 0, 0, 0

    for city in cities:
        city_file_count = len(city_files[city])
        if train_files + city_file_count <= train_count:
            train_cities.append(city)
            train_files += city_file_count
        elif val_files + city_file_count <= val_count:
            val_cities.append(city)
            val_files += city_file_count
        elif test_files + city_file_count <= test_count:
            test_cities.append(city)
            test_files += city_file_count
        else:
            break  # Stop once the counts are met

    # Ensure precise counts are achieved
    def adjust_split(target_count, current_count, source_split, target_split):
        for city in source_split[:]:  # Iterate over a copy of the list
            city_file_count = len(city_files[city])
            if current_count + city_file_count <= target_count:
                target_split.append(city)
                source_split.remove(city)
                current_count += city_file_count
            if current_count == target_count:
                break
        return current_count

    # Adjust splits if necessary
    train_files = adjust_split(train_count, train_files, cities, train_cities)
    val_files = adjust_split(val_count, val_files, cities, val_cities)
    test_files = adjust_split(test_count, test_files, cities, test_cities)

    # Helper function to copy files
    def copy_files(cities, split_type):
        for city in cities:
            for filename in city_files[city]:
                # Define corresponding filenames for all datasets
                files_to_copy = [
                    (os.path.join(images_2019, filename), os.path.join(output_dir, split_type, "Images", "T2019")),
                    (os.path.join(images_2024, filename.replace("_2019", "_2024")), os.path.join(output_dir, split_type, "Images", "T2024")),
                    (os.path.join(masks_2019, f"m_{filename}"), os.path.join(output_dir, split_type, "Masks", "T2019")),
                    (os.path.join(masks_2024, f"m_{filename.replace('_2019', '_2024')}"), os.path.join(output_dir, split_type, "Masks", "T2024")),
                    (os.path.join(cd1_output, f"cd1_m_{city}.tif"), os.path.join(output_dir, split_type, "cd1_Output")),
                    (os.path.join(cd2_output, f"cd2_m_{city}.tif"), os.path.join(output_dir, split_type, "cd2_Output")),
                    (os.path.join(cd3_output, f"cd3_m_{city}.tif"), os.path.join(output_dir, split_type, "cd3_Output")),
                ]

                # Copy each file to its destination
                for src, dest_dir in files_to_copy:
                    try:
                        if os.path.exists(src):
                            os.makedirs(dest_dir, exist_ok=True)
                            shutil.copy(src, os.path.join(dest_dir, os.path.basename(src)))
                        else:
                            print(f"File not found: {src}")  # Debugging missing files
                    except Exception as e:
                        print(f"Error copying {src}: {e}")

    # Copy files to train, val, and test folders
    copy_files(train_cities, "train")
    copy_files(val_cities, "val")
    copy_files(test_cities, "test")
    print("Dataset split complete.")

# Example usage
dataset_dir = "ChangeDetection"  # Your dataset directory
output_dir = "SplitDataset3"  # Output directory for train, val, and test splits
split_dataset(dataset_dir, output_dir, train_count=1070, val_count=230, test_count=230)


### Display the files in Dataset (Check if they match)

In [ ]:
import os
import matplotlib.pyplot as plt
import rasterio
import random
import numpy as np

def fetch_and_display_images(real_folder_2019, real_folder_2024,
                             input_folder_2019, input_folder_2024,
                             cd1_output_folder, cd2_output_folder, cd3_output_folder, index):
    """
    Fetch and display images divided into parts: mask_2019, mask_2024, cd1, cd2, cd3, real_2019, real_2024.
    """
    # Find the corresponding real and mask files
    real_2019_file = os.listdir(real_folder_2019)[index]
    # city_state = real_2019_file.replace("_2019.tif", "")
    city_state = real_2019_file[:-11]

    fig, axs = plt.subplots(4, 7, figsize=(25, 15))  # Adjust layout to fit all parts
    fig.suptitle(city_state)
    part_labels = ['1', '2', '3', '4']
    # print(city_state)

    for i, part in enumerate(part_labels):
        # Update file names with parts
        mask_2019_file = f"m_{city_state}_2019_{part}.tif"
        mask_2024_file = f"m_{city_state}_2024_{part}.tif"
        cd1_output = f"cd1_m_{city_state}_{part}.tif"
        cd2_output = f"cd2_m_{city_state}_{part}.tif"
        cd3_output = f"cd3_m_{city_state}_{part}.tif"
        real_2019_file = f"{city_state}_2019_{part}.tif"
        real_2024_file = f"{city_state}_2024_{part}.tif"

        # Paths to files
        mask_2019_path = os.path.join(input_folder_2019, mask_2019_file)
        mask_2024_path = os.path.join(input_folder_2024, mask_2024_file)
        cd1_output_path = os.path.join(cd1_output_folder, cd1_output)
        cd2_output_path = os.path.join(cd2_output_folder, cd2_output)
        cd3_output_path = os.path.join(cd3_output_folder, cd3_output)
        real_2019_path = os.path.join(real_folder_2019, real_2019_file)
        real_2024_path = os.path.join(real_folder_2024, real_2024_file)

        # Ensure files exist
        if not all(os.path.exists(p) for p in [mask_2019_path, mask_2024_path, cd1_output_path, cd2_output_path, cd3_output_path, real_2019_path, real_2024_path]):
            print(f"Some files for part {part} are missing.")
            continue
        # Read and display images for the current part
        with rasterio.open(mask_2019_path) as src:
            mask_2019 = src.read(1)
        with rasterio.open(mask_2024_path) as src:
            mask_2024 = src.read(1)
        with rasterio.open(cd1_output_path) as src:
            cd1_output_mask = src.read(1)
        with rasterio.open(cd2_output_path) as src:
            cd2_output_mask = src.read(1)
        with rasterio.open(cd3_output_path) as src:
            cd3_output_mask = src.read(1)
        with rasterio.open(real_2019_path) as src:
            real_2019 = np.dstack([src.read(band) for band in (1, 2, 3)])
        with rasterio.open(real_2024_path) as src:
            real_2024 = np.dstack([src.read(band) for band in (1, 2, 3)])

        # Display images
        axs[i, 0].imshow(real_2019 / real_2019.max())
        axs[i, 0].set_title(f"Real 2019 {part}")
        axs[i, 0].axis("off")

        axs[i, 1].imshow(real_2024 / real_2024.max())
        axs[i, 1].set_title(f"Real 2024 {part}")
        axs[i, 1].axis("off")

        axs[i, 2].imshow(mask_2019, cmap="viridis")
        axs[i, 2].set_title(f"Mask 2019 {part}")
        axs[i, 2].axis("off")

        axs[i, 3].imshow(mask_2024, cmap="viridis")
        axs[i, 3].set_title(f"Mask 2024 {part}")
        axs[i, 3].axis("off")

        axs[i, 4].imshow(cd1_output_mask, cmap="turbo")
        axs[i, 4].set_title(f"CD1 {part}")
        axs[i, 4].axis("off")

        axs[i, 5].imshow(cd2_output_mask, cmap="turbo")
        axs[i, 5].set_title(f"CD2 {part}")
        axs[i, 5].axis("off")

        axs[i, 6].imshow(cd3_output_mask, cmap="turbo")
        axs[i, 6].set_title(f"CD3 {part}")
        axs[i, 6].axis("off")
    plt.tight_layout()
    plt.show()

# Example usage:
base_folder = "ChangeDetectionDivided/Train"
real_folder_2019 = f"{base_folder}/Images/T2019"
real_folder_2024 = f"{base_folder}/Images/T2024"
input_folder_2019 = f"{base_folder}/Masks/T2019"
input_folder_2024 = f"{base_folder}/Masks/T2024"
cd1_output_folder = f"{base_folder}/cd1_Output"
cd2_output_folder = f"{base_folder}/cd2_Output"
cd3_output_folder = f"{base_folder}/cd3_Output"

for i in range(20):
    j = random.randint(0, len(os.listdir(input_folder_2019)) - 1)
    fetch_and_display_images(real_folder_2019, real_folder_2024,
                             input_folder_2019, input_folder_2024,
                             cd1_output_folder, cd2_output_folder, cd3_output_folder, j)

#### Debugging

In [ ]:
def debug_file_names(real_folder_2019, real_folder_2024,
                     input_folder_2019, input_folder_2024,
                     cd1_output_folder, cd2_output_folder, cd3_output_folder, index):
    """
    Print the corrected file names for debugging.
    """
    # Get the real 2019 file and extract city_state correctly
    real_2019_file = os.listdir(real_folder_2019)[index]
    # city_state = real_2019_file.replace("_2019_1.tif", "")  # Remove extension here
    part_labels = ['1', '2', '3', '4']
    city_state = real_2019_file[:-11]
    # print(real_2019_file, city_state)
    
    for part in part_labels:
        # Construct file names with parts
        mask_2019_file = f"m_{city_state}_2019_{part}.tif"
        mask_2024_file = f"m_{city_state}_2024_{part}.tif"
        cd1_output = f"cd1_m_{city_state}_{part}.tif"
        cd2_output = f"cd2_m_{city_state}_{part}.tif"
        cd3_output = f"cd3_m_{city_state}_{part}.tif"
        real_2019_file = f"{city_state}_2019_{part}.tif"
        real_2024_file = f"{city_state}_2024_{part}.tif"

        # Paths to files
        mask_2019_path = os.path.join(input_folder_2019, mask_2019_file)
        mask_2024_path = os.path.join(input_folder_2024, mask_2024_file)
        cd1_output_path = os.path.join(cd1_output_folder, cd1_output)
        cd2_output_path = os.path.join(cd2_output_folder, cd2_output)
        cd3_output_path = os.path.join(cd3_output_folder, cd3_output)
        real_2019_path = os.path.join(real_folder_2019, real_2019_file)
        real_2024_path = os.path.join(real_folder_2024, real_2024_file)

        # Print file paths
        print(f"Part {part}:")
        print(f"  Mask 2019: {mask_2019_path}")
        print(f"  Mask 2024: {mask_2024_path}")
        print(f"  CD1: {cd1_output_path}")
        print(f"  CD2: {cd2_output_path}")
        print(f"  CD3: {cd3_output_path}")
        print(f"  Real 2019: {real_2019_path}")
        print(f"  Real 2024: {real_2024_path}")
        print()


base_folder = "CDTestDivided/Train"
real_folder_2019 = f"{base_folder}/Images/T2019"
real_folder_2024 = f"{base_folder}/Images/T2024"
input_folder_2019 = f"{base_folder}/Masks/T2019"
input_folder_2024 = f"{base_folder}/Masks/T2024"
cd1_output_folder = f"{base_folder}/cd1_Output"
cd2_output_folder = f"{base_folder}/cd2_Output"
cd3_output_folder = f"{base_folder}/cd3_Output"

# Debug file names for a random index
random_index = random.randint(0, len(os.listdir(real_folder_2019)) - 1)
debug_file_names(real_folder_2019, real_folder_2024,
                 input_folder_2019, input_folder_2024,
                 cd1_output_folder, cd2_output_folder, cd3_output_folder, random_index)
